In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Customized Subfunctions

``AUCMEDI`` offers the possibility to implement customized subfunctions for (additional, non-standard) preprocessing of the input images.  

In this notebook we will implement two customized subfunctions: a function that adds 1 to each pixel value and a function that rotates the input image.

But first, we need to download the data.

## Downloading the data

In [2]:
from pathlib import Path
import wget
import zipfile

cwd = !pwd
datadir = cwd[0] + "/data"
Path(datadir).mkdir(parents=True, exist_ok=True)

#print('Beginning file download with wget module')

#url = 'https://zenodo.org/record/53169/files/Kather_texture_2016_image_tiles_5000.zip?download=1'
#wget.download(url, datadir)

#with zipfile.ZipFile("data/Kather_texture_2016_image_tiles_5000.zip","r") as zip_ref:
#    zip_ref.extractall("data")

from aucmedi.data_processing.io_data import input_interface
ds_loader = input_interface("directory", path_imagedir="data/Kather_texture_2016_image_tiles_5000", path_data=None, training=True, ohe=False)
(samples, class_ohe, nclasses, class_names, image_format) = ds_loader

from aucmedi.sampling.split import sampling_split
train, validation, test = sampling_split(samples, class_ohe, sampling=[0.5, 0.25, 0.25], 
                                         stratified=True, iterative=False, seed=123)

2022-08-14 12:35:09.514247: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## Define the customized Subfunctions

Now we are going to define our customized subfunctions.  

All subfunctions are based on the abstract base class [Subfunction_Base](https://frankkramer-lab.github.io/aucmedi/reference/data_processing/subfunctions/sf_base/#aucmedi.data_processing.subfunctions.sf_base.Subfunction_Base).

All classes that are derived from the abstract class [Subfunction_Base](https://frankkramer-lab.github.io/aucmedi/reference/data_processing/subfunctions/sf_base/#aucmedi.data_processing.subfunctions.sf_base.Subfunction_Base) are required to have the functions ``__init__()`` and ``transform()``. 

In [3]:
from aucmedi.data_processing.subfunctions.sf_base import Subfunction_Base
import numpy as np

class plus_one(Subfunction_Base):
    def __init__(self): 
        pass

    def transform(self, image):
        new_images = np.where(image < 255, image, + 1.0) 
        return new_images                

    
class rotate(Subfunction_Base):
    def __init__(self):    
        pass
    
    def transform(self, image):
        new_images = np.rot90(image, 2)    # rotate 180 degrees
        return new_images   

## Define the model

The definition of the ``NeuralNetwork`` works as usual. If you have questions, have a look in the corresponding notebook.

In [4]:
from aucmedi.neural_network.model import NeuralNetwork
import tensorflow.keras as tfa

f1Score = tfa.metrics.F1Score(num_classes=nclasses, threshold=0.5)

model = NeuralNetwork(n_labels=nclasses, channels=3, architecture="2D.ResNet50", 
                      loss="categorical_crossentropy", metrics=["categorical_accuracy", f1Score], 
                      activation_output="softmax", pretrained_weights=False)

2022-08-14 12:35:12.469111: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-14 12:35:13.027895: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22844 MB memory:  -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:3f:00.0, compute capability: 7.5


## Train the model

Now we created batches for training with the `DataGenerator`.  
This is, were we can call our subfunction. We tell the ``DataGenerator`` with the ``subfunction``s-argument to apply the transform-method of the ``plus_one()`` and the ``rotate()`` object on each image. (The objects have to be put is a list, even if it is only one object.)

Next, the model can be trained as usual.

In [5]:
from aucmedi.data_processing.data_generator import DataGenerator

train_generator = DataGenerator(samples=train[0], path_imagedir="data/Kather_texture_2016_image_tiles_5000",
                                               resize=model.meta_input, standardize_mode=model.meta_standardize,
                                               labels=train[1], image_format=image_format, batch_size=32, data_aug=None, 
                                               grayscale=False, prepare_images=False, subfunctions=[plus_one(), rotate()],
                                               sample_weights=None, seed=123, workers=1)
val_generator = DataGenerator(samples=validation[0], path_imagedir="data/Kather_texture_2016_image_tiles_5000",
                                             resize=model.meta_input, standardize_mode=model.meta_standardize,
                                             labels=validation[1], image_format=image_format, batch_size=32, data_aug=None, 
                                             grayscale=False, prepare_images=False, subfunctions=[plus_one(), rotate()],
                                             sample_weights=None, seed=123, workers=1)

history = model.train(training_generator=train_generator, validation_generator=val_generator, epochs=20, iterations=None, 
                                         callbacks=None, class_weights=None, transfer_learning=False)

Epoch 1/20


2022-08-14 12:35:19.992075: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-08-14 12:35:20.536104: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


79/79 [==============================] - 23s 201ms/step - loss: 1.9261 - categorical_accuracy: 0.4420 - f1_score: 0.4283 - val_loss: 3.3297 - val_categorical_accuracy: 0.1248 - val_f1_score: 0.0279
Epoch 2/20
79/79 [==============================] - 15s 188ms/step - loss: 1.3214 - categorical_accuracy: 0.5984 - f1_score: 0.5840 - val_loss: 8.4594 - val_categorical_accuracy: 0.1248 - val_f1_score: 0.0277
Epoch 3/20
79/79 [==============================] - 15s 188ms/step - loss: 1.0543 - categorical_accuracy: 0.6444 - f1_score: 0.6299 - val_loss: 5.6121 - val_categorical_accuracy: 0.1760 - val_f1_score: 0.0717
Epoch 4/20
79/79 [==============================] - 14s 180ms/step - loss: 1.0774 - categorical_accuracy: 0.6520 - f1_score: 0.6446 - val_loss: 2.1746 - val_categorical_accuracy: 0.4784 - val_f1_score: 0.4117
Epoch 5/20
79/79 [==============================] - 15s 194ms/step - loss: 0.9084 - categorical_accuracy: 0.7064 - f1_score: 0.7075 - val_loss: 2.2169 - val_categorical_accura